## Running Queries

Here we load a random paper, search for nearest neighbors with RediSearch, and compare the title/abstract of the two papers.

In [ ]:
import numpy as np

from redis.asyncio import Redis
from utils.search_index import SearchIndex
from time import time

In [ ]:
redis_conn = await Redis(host='redis', port='6379')
search_index = SearchIndex('papers', redis_conn)

In [ ]:
# Fetch one random paper from Redis to test
async def fetch_one(redis_conn):
    async for key in redis_conn.scan_iter("paper:*"):
        return await redis_conn.hgetall(key)

In [ ]:
paper = await fetch_one(redis_conn)

In [ ]:
paper[b'id']

In [ ]:
# Vector created in the last step!
np.frombuffer(paper[b'vector'])

In [ ]:
# Construct search
async def search(redis_conn, search_index, k: int, paper_key: str):
    query = search_index.vector_query(
            categories = [],
            years = [],
            number_of_results = k
    )
    print("searching for nearest neighbors to", paper_key)
    start = time()
    vector = await redis_conn.hget(paper_key, "vector")
    result = await redis_conn.ft(search_index.index_name).search(
        query,
        query_params={"vec_param": vector}
    )
    print("done in", time()-start, "seconds")
    return result

In [ ]:
paper_key = "paper:" + paper[b"id"].decode('utf-8')
result = await search(redis_conn, search_index, 5, paper_key)

In [ ]:
result

In [ ]:
result.docs[1]

In [ ]:
# Input document
await redis_conn.hmget(paper_key, "title", "abstract")

In [ ]:
# Nearest neighbor result
await redis_conn.hmget(result.docs[1].id, "title", "abstract")